In [2]:
import numpy as np
import pandas as pd
import warnings
import re
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
def convert_size(size):
    if isinstance(size, str):
        if 'k' in size:
            return float(size.replace('k', "")) * 1024
        elif 'M' in size:
            return float(size.replace('M', "")) * 1024 * 1024
        elif 'Varies with device' in size:
            return np.nan
    return size

In [4]:
def clean_installs(df):
    df['Installs'] = df['Installs'].apply(lambda x: x.replace('+', "") if '+' in str(x) else x)
    df['Installs'] = df['Installs'].apply(lambda x: x.replace(',', "") if ',' in str(x) else x)
    df['Installs'] = df['Installs'].apply(lambda x: int(x))
    return df

In [5]:
def bin_categories(df):
    bins = [-1, 0, 10, 1000, 10000, 100000, 1000000, 10000000, 10000000000]
    labels=['no', 'Very low', 'Low', 'Moderate', 'More than moderate', 'High', 'Very High', 'Top Notch']
    df['Installs_category'] = pd.cut(df['Installs'], bins=bins, labels=labels)
    return df

In [6]:
def clean_price(df):
    df['Price'] = df['Price'].apply(lambda x: x.replace('$', '') if '$' in str(x) else x)
    df['Price'] = df['Price'].apply(lambda x: float(x))
    return df

In [7]:
def specific_clean(df):
    df.drop_duplicates(inplace = True)
    df = df.drop(df[df['Category'] == '1.9'].index)
    df.loc[9148, 'Type'] = 'Free'
    df = df.drop(df[df['Android Ver'].isnull()].index)
    return df

In [8]:
def clean_data(file_path):
    # load csv into df
    df = pd.read_csv(file_path)

    # clean specific values for this table
    df = specific_clean(df)
    df.rename(columns={
    'Last Updated': 'Last_Updated',
    'Android Ver': 'Android_Ver',
    'Content Rating' : 'Content_Rating'
    }, inplace=True)
    # drop name duplicates and keep the record with highest nb of reviews
    df = df.loc[df.groupby('App')['Reviews'].idxmax()]
    
    # clean size
    df['Size'] = df['Size'].apply(convert_size)
    df['Size'] = df['Size'].apply(lambda x: (x/(1024*1024)))
    df.rename(columns={'Size': "Size_in_MB"}, inplace=True)
    df['Size_in_MB'] = df['Size_in_MB'].round(2)

    # clean installs
    df = clean_installs(df)
    
    # binning categories
    df = bin_categories(df)
    
    # clean price
    df = clean_price(df)
    
    # fill null ratings
    mean_by_cat = df.groupby('Category')['Rating'].transform('mean')
    df['Rating'] = df['Rating'].fillna(mean_by_cat)
    df['Rating'] = df['Rating'].round(2)
    
    # drop some values
    df.dropna(subset = ['Size_in_MB', 'Current Ver'], inplace=True)
    
    # concert datatypes
    df['Reviews'] = df['Reviews'].astype('int')
    df['Last_Updated'] = pd.to_datetime(df['Last_Updated'])

    return df

#clean_data('googleplaystore.csv')

In [9]:
def clean_reviews(file_path):
    df = pd.read_csv(file_path)
    df = df.dropna()
    mean_sentiment = df.groupby('App')['Sentiment_Polarity'].mean()
    cols = ['Translated_Review', 'Sentiment', 'Sentiment_Polarity', 'Sentiment_Subjectivity']
    df = df.drop(columns=cols, errors='ignore')
    new_df = df.copy()
    new_df['Mean_App_Sentiment'] = new_df['App'].map(mean_sentiment)
    new_df['Mean_App_Sentiment'] = new_df['Mean_App_Sentiment'].round(2)
    return new_df

In [10]:
def combine_tables():
    df1 = clean_data('googleplaystore.csv')
    df2 = clean_reviews('googleplaystore_user_reviews.csv')
    df_merged = pd.merge(df1, df2, on='App', how='left')
    df_merged['Mean_App_Sentiment'].fillna(0, inplace=True)
    return df_merged
    

In [11]:
df = combine_tables()
df[df['Mean_App_Sentiment'].notnull()]

,App,Category,Rating,Reviews,Size_in_MB,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Current Ver,Android_Ver,Installs_category,Mean_App_Sentiment
0,"""i DT"" Fútbol. Todos Somos Técnicos.",SPORTS,4.22,27,3.6,500,Free,0.00,Everyone,Sports,2017-10-07,0.22,4.1 and up,Low,0.0
1,+Download 4 Instagram Twitter,SOCIAL,4.50,40467,22.0,1000000,Free,0.00,Everyone,Social,2018-08-02,5.03,4.1 and up,High,0.0
2,- Free Comics - Comic Apps,COMICS,3.50,115,9.1,10000,Free,0.00,Mature 17+,Comics,2018-07-13,5.0.12,5.0 and up,Moderate,0.0
3,.R,TOOLS,4.50,259,0.2,10000,Free,0.00,Everyone,Tools,2014-09-16,1.1.06,1.5 and up,Moderate,0.0
4,/u/app,COMMUNICATION,4.70,573,53.0,10000,Free,0.00,Mature 17+,Communication,2018-07-03,4.2.4,4.1 and up,Moderate,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31952,"뽕티비 - 개인방송, 인터넷방송, BJ방송",VIDEO_PLAYERS,4.04,414,59.0,100000,Free,0.00,Mature 17+,Video Players & Editors,2018-07-18,4.0.7,4.0.3 and up,More than moderate,0.0
31953,💎 I'm rich,LIFESTYLE,3.80,718,26.0,10000,Paid,399.99,Everyone,Lifestyle,2018-03-11,1.0.0,4.4 and up,Moderate,0.0
31954,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.60,22098,18.0,1000000,Free,0.00,Everyone,Social,2018-07-24,4.2.4,4.0.3 and up,High,0.0
31955,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.00,19,3.2,10000,Free,0.00,Everyone,Tools,2017-10-21,1.0,4.2 and up,Moderate,0.0


In [12]:
df['Mean_App_Sentiment']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
31952    0.0
31953    0.0
31954    0.0
31955    0.0
31956    0.0
Name: Mean_App_Sentiment, Length: 31957, dtype: float64

In [13]:
def clean_android_version(version):
    
    numbers = re.findall(r'\d+(?:\.\d+)?', str(version))
    if numbers:
        # Convert all found numbers to float and return the minimum
        return min(float(num) for num in numbers)
    else:
        return None  # or you could return a default value

In [14]:
# Apply the function to the column
df['Android_Ver'] = df['Android_Ver'].apply(clean_android_version)

# Convert to float and handle any remaining NaN values
df['Android_Ver'] = df['Android_Ver'].astype(float).fillna(df['Android_Ver'].mean())

In [15]:
df['App'].duplicated().sum()

23535

In [16]:
df[df['App'].duplicated(keep=False)]['App'].nunique()

551

In [17]:
df= df.drop_duplicates()

In [18]:
final_df = df[['App','Category','Size_in_MB','Type','Price','Content_Rating','Genres','Last_Updated','Android_Ver']]
final_df['Android_Ver']
final_df.to_csv('divided_datasets_1/finaldataset.csv', index = False)

In [19]:
shuffled_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)
df_parts = np.array_split(shuffled_df, 3)
df1, df2, df3 = df_parts

In [20]:
#saving each part of the dataset to a new csv
df1.to_csv('divided_datasets_1/df1.csv', index=False)
df2.to_csv('divided_datasets_1/df2.csv', index=False)
df3.to_csv('divided_datasets_1/df3.csv', index=False)

In [21]:
final2_df = df[['App','Category','Rating','Reviews','Size_in_MB','Type','Price','Content_Rating','Genres','Last_Updated','Android_Ver']]
final2_df['Android_Ver']
final2_df.to_csv('divided_dataset_2/final_2_dataset.csv', index = False)
shuffled_df = final2_df.sample(frac=1, random_state=42).reset_index(drop=True)
df_parts = np.array_split(shuffled_df, 3)
df2_1, df2_2, df2_3 = df_parts

In [22]:
df2_1.to_csv('divided_dataset_2/df2_1.csv', index=False)
df2_2.to_csv('divided_dataset_2/df2_2.csv', index=False)
df2_3.to_csv('divided_dataset_2/df2_3.csv', index=False)